## Introduction

`Idefix` is bundled with its own python routine to read its output files. The VTK files produced by Idefix follow the VTK standard and can therefore be opened with any VTK reader like Paraview of Visit.

In this tutorial, we will learn how to: 
- load an IDEFIX dataset 
- visualize the data
- modify the data to plot new quantities

## Visualization from an Idefix point of view

### 1 - Loading a dataset

Get the path of the directory where you ran the planet-disk interaction problem (`../AdvancedSetup/problem1/`), and import the following libraries (we import the functions included in your clone of the idefix repository, so `$IDEFIX_DIR` must be set)

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

import os
import sys
sys.path.append(os.getenv("IDEFIX_DIR"))
from pytools.vtk_io import readVTK

In [ ]:
directory = "../../AdvancedSetup/problem1/"  # CHECK YOUR PATH!
# load the VTK file data.0003.vtk in the dataset V
V = readVTK(directory+"data.0003.vtk")

`V` is a class that contains the following attributes

In [ ]:
print(vars(V).keys())

Of particular importance is the dictionnary `data` that contains all of the 3D fields produced by the code:

In [ ]:
print(V.data.keys())

### 2 - Compute a new field

We now want to consider the azimuthal component of the gas velocity ($v_\phi$) computed in the frame corotating with the planet  (located in $R_p$, rotating at $\Omega_p=R_p^{-3/2}$), and therefore create a corrotating field $v_\phi^C$

$v_\phi^C = v_\phi - R\Omega_p$

We start by definiting the planet's parameters

In [ ]:
# radial location of the planet
Rp = 1.0
# Angular velocity of the planet
OmegaP = Rp**(-3/2)

We then compute the new azimuthal component and store it in our dataSet. `V_\phi` is named `VX2`, so name the new field `VX2c`: 

In [ ]:
# The radius is the first coordinate of the simulation (x1), so it is V.x in the dataset
# 3D array of the radial coordinate
R = V.x[:, None, None]

V.data['VX2c']=V.data["VX2"]-OmegaP*R

### 3 - Visualize the data: 1D cut

We can perform simple 1D cut of the dataset using standard matplolib calls.

In [ ]:
# Density cut vs R at phi=0 (planet location)
plt.figure(figsize=(7,5))
plt.loglog(V.x,V.data['RHO'][:,0,0],label='@planet location')
plt.loglog(V.x,V.data['RHO'][:,V.ny//2,0],label='opposite planet')
plt.legend()
plt.xlabel('R')
plt.ylabel(r'$\Sigma$')

In [ ]:
# Density cut vs phi at phi=0 (planet location)
iplanet=np.argwhere(V.x>=1.0)[0][0]
plt.figure(figsize=(7,5))
plt.plot(V.y,V.data['VX1'][iplanet,:,0],'.',label='$V_r$')
plt.plot(V.y,V.data['VX2c'][iplanet,:,0],'.',label='$\delta V_\phi$')
plt.legend()
plt.xlabel('$\phi$')
plt.ylabel(r'velocity')

### 4 - Visualize the data: 2D cut

Before plotting 2D data, we need to convert the field into an object that can be plotted. To do that, we need to map the coordinate system stored in the VTK file (`r=V.x`, `y=V.phi`) to a cartesian system. We will need both the cell center coordinates, and the cell corners.

In [ ]:
# Corner of the cells, deduced from the left side of the cells in R and phi, stored in V.xl and V.yl
x=V.xl[:,None]*np.cos(V.yl[None,:])
y=V.xl[:,None]*np.sin(V.yl[None,:])

# Center of the cells, deduced from the central coordinate of each cell
xc=V.x[:,None]*np.cos(V.y[None,:])
yc=V.x[:,None]*np.sin(V.y[None,:])

Note that here there is just one cell in the vertical direction, so you don't need to perform a vertical slice beforehand. But in a general 3D case, you need first to reduce the dimension with one or two operations, and then map the field in the target plane.

In [ ]:
import matplotlib.colors as colors

# plot the density using log colorbar
plt.figure(figsize=(7,7))
plt.pcolormesh(x,y,V.data['RHO'][:,:,0],cmap='inferno',norm=colors.LogNorm())
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.title('Density')
plt.gca().set_aspect('equal')

We have here a cartesian view of the density field, in log scale, using the colormap `"inferno"` and with a colorbar and title. We can do the same in a polar view.

Let's do the same for the velocity field:

In [ ]:
plt.figure(figsize=(7,7))
plt.pcolormesh(x,y,V.data['VX1'][:,:,0],cmap='seismic',vmin=-0.1,vmax=0.1)
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.title('$V_r$')
plt.gca().set_aspect('equal')

plt.figure(figsize=(7,7))
plt.pcolormesh(x,y,V.data['VX2c'][:,:,0],cmap='seismic',vmin=-0.5,vmax=0.5)
plt.xlabel('x')
plt.ylabel('y')
plt.colorbar()
plt.title('$V_\phi-\Omega_p R$')
plt.gca().set_aspect('equal')

Exercise: plot the gas to dust density ratio in a polar view!